## Consultas a la base de datos

In [2]:
!pip install pymysql

In [1]:
import pymysql

## Crear la BD desde 0

In [75]:
Users_table = "CREATE TABLE `users` (`Nomina` varchar(10) NOT NULL,`Nombre` varchar(100) NOT NULL,`Puesto` varchar(50) NOT NULL,`Tags` text NOT NULL,`Certificado` blob,`Estatus` varchar(20) NOT NULL, PRIMARY KEY (`Nomina`))"
Documentos_table = "CREATE TABLE `documentos` (`Hash` varchar(80) NOT NULL,`Tipo` varchar(100) NOT NULL,`Nombre` varchar(100) NOT NULL,`Descripcion` text NOT NULL,`Tags` varchar(100) NOT NULL,`Estatus` varchar(10) NOT NULL,`TIME` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,PRIMARY KEY (`Hash`))"
Firmas_table = "CREATE TABLE `firmas` (`ID` int(11) NOT NULL AUTO_INCREMENT,`Hash` varchar(100) NOT NULL,`Doc_signed` blob,`Nomina` varchar(50) NOT NULL,`TIME` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP, PRIMARY KEY (`ID`))"

In [84]:
connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "sys"
        )
cursor = connection.cursor()
cursor.execute("CREATE DATABASE teleton")
cursor.close()

connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
cursor = connection.cursor()

In [85]:
cursor.execute(Firmas_table)
cursor.execute(Documentos_table)
cursor.execute(Users_table)

0

In [78]:
cursor.close()

In [2]:
class DataBase:
    def __init__(self):
        self.connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
        self.cursor = self.connection.cursor()
    
        print("Conexión exitosa")
    
    def insert_users(self, nomina, nombre, puesto, tags, certificado, estatus):
            certificado = open(certificado, "rb").read()
            sql = """INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES (%s,%s,%s,%s,%s,%s)"""
            try:
                self.cursor.execute(sql, (nomina, nombre, puesto, tags, certificado, estatus))
                print(f"{nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def insert_documentos(self, Hash, Tipo, Nombre, Descripcion, Tags, Estatus):
            sql = f"INSERT INTO documentos (`Hash`, `Tipo`, `Nombre`, `Descripcion`, `Tags`, `Estatus`) VALUES ('{Hash}', '{Tipo}', '{Nombre}', '{Descripcion}', '{Tags}', '{Estatus}')"
            try:
                self.cursor.execute(sql)
                print(f"{Nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def insert_firma(self, Hash, Nomina): 
            #Doc_signed = open(Doc_signed, "rb").read()
            sql = """ INSERT INTO firmas  (Hash, Nomina) VALUES (%s,%s)"""
            try:
                self.cursor.execute(sql, (Hash, Nomina))
                print(f"{Hash} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
    
    def cargar_firma(self, Doc_signed, Hash, Nomina):
        Doc_signed = open(Doc_signed, "rb").read()
        sql = """ UPDATE firmas SET Doc_signed = %s WHERE (Hash, Nomina) = (%s, %s)"""
        self.cursor.execute(sql, (Doc_signed, Hash, Nomina))
                
    def select(self, tabla, what, where, value, where_2 = "", value_2 = ""):
        if what == "*":
            sql = f"SELECT {what} FROM {tabla}"
        elif where_2 != "":
            sql = f"SELECT {what} FROM {tabla} where {where} = '{value}' AND {where_2} = '{value_2}'"
            
        else:
            sql = f"SELECT {what} FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        val = self.cursor.fetchall()
        return val
        
    def update(self, tabla, what, value, where_col, where_val):
        sql = f"UPDATE {tabla} set {what} = '{value}' where {where_col} = '{where_val}'"
        self.cursor.execute(sql)
        print("Actualización exitosa")
        
    def delete(self, tabla, where, value):
        sql = f"DELETE FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        print("Eliminación exitosa")
    

database = DataBase()

Conexión exitosa


In [70]:
database.cargar_firma(Doc_signed = "nombreDoc_nomina_fecha20220419_102737.sign",
                     Hash = "A01750443",
                     Nomina = "IDM")

### Insertar un usuario

In [35]:
database.insert_users(nomina = "A01752080",
                nombre = "Jairo",
                puesto = "Estudiante",
                tags = "IDM",
                certificado = "Ejemplo/CertificadoJairo_Enrique.crt",
                estatus = "Activo")

Jairo ha sido añadido a la base de datos.


### Insertar un documento

In [73]:
database.insert_documentos(Hash = "jxajdljdljswss", 
                           Tipo = "Certificado", 
                           Nombre = "Ventas.pdf", 
                           Descripcion = "Documento que bla bla",
                           Tags = "Ventas", 
                           Estatus = "Activo")

Ventas.pdf ha sido añadido a la base de datos.


### Insertar una firma

In [63]:
database.insert_firma(Hash = "A01750443",
                #Doc_signed = "Ejemplo/CertificadoJairo_Enrique.crt",
                #Doc_signed = "nombreDoc_nomina_fecha20220419_110201.sign",
                Nomina = "IDM")

A01750443 ha sido añadido a la base de datos.


### Ejemplo de select

In [118]:
#"""
database.select(tabla = "documentos", 
                what = "*", 
                where = "ID", 
                value = "1")
#"""

(('jxajdljdljswss',
  'Certificado',
  'Ventas.pdf',
  'Documento que bla bla',
  'Ventas',
  'Activo',
  datetime.datetime(2022, 4, 19, 12, 9, 55)),
 ('<sha256 HASH object @ 0x000001E3BD9C9230>',
  'Libro',
  'Doc_1.pdf',
  'Justine o los infortunios de la virtud',
  'Marqués de Sade',
  'Activo',
  datetime.datetime(2022, 4, 19, 12, 10, 20)),
 ('50df68716b0074ebd4e5f2ccb5b378d89a8a174128394a57654d4410d4a268e5',
  'Libro',
  'Doc_1.pdf',
  'Justine o los infortunios de la virtud',
  'Marqués de Sade',
  'Activo',
  datetime.datetime(2022, 4, 19, 12, 10, 54)))

In [119]:
F = database.select(tabla = "firmas", 
                what = "ID, Nomina, Doc_signed", 
                where = "Hash", 
                value = "A01750443")
F

()

### Update

In [55]:
database.update(tabla = "users", 
                what = "Nombre", 
                value="Juanito", 
                where_col = "Nomina", 
                where_val = "A01752068")

Actualización exitosa


### Delete

In [32]:
database.delete(tabla = "firmas", 
                where = "ID", 
                value="2")

Eliminación exitosa


## Generación del certificado digital y llave privada

In [5]:
from OpenSSL import crypto, SSL

def cert_gen(
    emailAddress="emailAddress",
    commonName="commonName",
    countryName="NT",
    localityName="localityName",
    stateOrProvinceName="stateOrProvinceName",
    organizationName="organizationName",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=10*365*24*60*60,
    KEY_FILE = "private.key",
    CERT_FILE="selfsigned.crt"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in selfsigned.crt -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha256')
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [9]:
cert_gen(emailAddress="A01750443@tec.mx",
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "privateJERS_2.key",
    CERT_FILE="CertificadoJERS_2.crt")

## Cargar el certificado

In [30]:
import OpenSSL.crypto 
cert = OpenSSL.crypto.load_certificate(
    OpenSSL.crypto.FILETYPE_PEM, 
    open('CertificadoJERS_2.crt').read()
)
cert.get_notBefore()

b'20220418135149Z'

### Obteniendo valores del certificado

In [31]:
print(cert.get_signature_algorithm()) #Con qué algoritmo fue firmado
print(cert.get_notAfter()) #fecha de expiración
print(cert.get_pubkey())
print(cert.get_subject()) # Datos sobre el certificado

b'sha256WithRSAEncryption'
b'20230418135149Z'
<X509Name object '/C=MX/ST=Nuevo Le\xC3\xB3n/L=Monterrey/O=Tecnol\xC3\xB3gico de Monterrey/OU=organizationUnitName/CN=tec.mx/emailAddress=A01750443@tec.mx'>


## Revisar si un certificado coincide con la llave privada

In [4]:
import OpenSSL.crypto 
def check_associate_cert_with_private_key(cert, private_key):
    """
    :type cert: str
    :type private_key: str
    :rtype: bool
    """
    try:
        private_key_obj = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, open(private_key).read())
    except OpenSSL.crypto.Error:
        raise Exception('private key is not correct: %s' % private_key)

    try:
        cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)
        #cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, open(cert).read())
    except OpenSSL.crypto.Error:
        raise Exception('certificate is not correct: %s' % cert)

    context = OpenSSL.SSL.Context(OpenSSL.SSL.TLSv1_METHOD)
    context.use_privatekey(private_key_obj)
    context.use_certificate(cert_obj)
    try:
        context.check_privatekey()
        return True
    except OpenSSL.SSL.Error:
        return False

In [40]:
check_associate_cert_with_private_key("CertificadoJERS.crt", "privateJERS.key")

True

## Hash de cualquier documento

In [3]:
import sys
import hashlib

def Hash_document(file):
    # BUF_SIZE is totally arbitrary, change for your app!
    BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

    md5 = hashlib.md5()
    sha1 = hashlib.sha256()
    #hashlib.sha512

    with open(file, 'rb') as f:
        while True:
            data = f.read(BUF_SIZE)
            if not data:
                break
            md5.update(data)
            sha1.update(data)

    #print("MD5: {0}".format(md5.hexdigest()))
    #print("SHA1: {0}".format(sha1.hexdigest()))
    return sha1

## Generando y verificando un certificado

In [88]:
import signVerify
sha1 = Hash_document("../Ex.pdf")
file_name = signVerify.gen_signature("privateJERS.key", bytes(sha1.hexdigest(), 'utf-8'), "s", "s")

In [26]:
#file_name = "nombreDoc_nomina_fecha20220419_102737.sign"
signVerify.verify("certificadoJERS.crt", bytes(sha1.hexdigest(), 'utf-8'), file_name)

Verification successful


In [27]:
signVerify.verify("certificadoJERS_2.crt", bytes(sha1.hexdigest(), 'utf-8'), file_name)

Verification failed


In [89]:
#signVerify.verify("certificadoJERS.crt", bytes(sha1.hexdigest(), 'utf-8'), F[0][2], load = False)
file_name

's_s_20220419_204735.sign'

# Ejemplo de uso del sistema

## Creando usuarios

### Creando certificados

In [6]:
nombre_1 = "Jairo Enrique"
nomina_1 = "A01750443"
email_1 = "A01750443@tec.mx"

nombre_2 = "Ramirez Sanchez"
nomina_2 = "A01752068"
email_2 = "A01752068@tec.mx"


cert_gen(emailAddress=email_1,
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "Ejemplo/private"+nomina_1.replace(" ","_")+".key",
    CERT_FILE="Ejemplo/Certificado"+nomina_1.replace(" ","_")+".crt")

cert_gen(emailAddress=email_2,
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "Ejemplo/private"+nomina_2.replace(" ","_")+".key",
    CERT_FILE="Ejemplo/Certificado"+nomina_2.replace(" ","_")+".crt")

print("Firmas generadas")
print("Guarde su llave privada en un lugar aparte")
print("-------------------\n")
print("Subiendo certificado a la base de datos")

database.insert_users(nomina = nomina_1,
                nombre = nombre_1,
                puesto = "Estudiante",
                tags = "IDM",
                certificado = f"Ejemplo/Certificado{nomina_1}.crt",
                estatus = "Activo")

database.insert_users(nomina = nomina_2,
                nombre = nombre_2,
                puesto = "Estudiante",
                tags = "IDM",
                certificado = f"Ejemplo/Certificado{nomina_2}.crt",
                estatus = "Activo")

Firmas generadas
Guarde su llave privada en un lugar aparte
-------------------

Subiendo certificado a la base de datos
Jairo Enrique ha sido añadido a la base de datos.
Ramirez Sanchez ha sido añadido a la base de datos.


## Solicitar firma

In [7]:
#Se sube el documento que se quiere firmar
file_1 = "Ejemplo/Doc_1.pdf"

database.insert_documentos(Hash = Hash_document(file_1).hexdigest(), 
                           Tipo = "Libro", 
                           Nombre = "Doc_1.pdf", 
                           Descripcion = "Justine o los infortunios de la virtud",
                           Tags = "Marqués de Sade",
                           Estatus = "Activo")

file_2 = "Ejemplo/Doc_2.pdf"

database.insert_documentos(Hash = Hash_document(file_2).hexdigest(), 
                           Tipo = "Libro", 
                           Nombre = "Doc_2.pdf", 
                           Descripcion = "Suenio de una noche de verano",
                           Tags = "Shakespeare",
                           Estatus = "Activo")

Doc_1.pdf ha sido añadido a la base de datos.
Doc_2.pdf ha sido añadido a la base de datos.


In [8]:
#Se solicitan las firmas de los empleados
database.insert_firma(Hash = Hash_document(file_1).hexdigest(),
                Nomina = nomina_1)

database.insert_firma(Hash = Hash_document(file_1).hexdigest(),
                Nomina = nomina_2)

database.insert_firma(Hash = Hash_document(file_2).hexdigest(),
                Nomina = nomina_1)

50df68716b0074ebd4e5f2ccb5b378d89a8a174128394a57654d4410d4a268e5 ha sido añadido a la base de datos.
50df68716b0074ebd4e5f2ccb5b378d89a8a174128394a57654d4410d4a268e5 ha sido añadido a la base de datos.
529408553f25564439f509c3554f3ad705848bff071f5342825f20a051c29264 ha sido añadido a la base de datos.


## Generar el archivo firmado

In [9]:
import signVerify

In [10]:
#Falta verificar si la privada coincide y su certificado está vigente

In [11]:
from datetime import datetime
#Verificando si todavía es vigente
def VerificarVigencia(Certificado_1):
    try:
        cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, Certificado_1)
        #cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, open(cert).read())
    except OpenSSL.crypto.Error:
        raise Exception('certificate is not correct: %s' % cert)

    fecha = cert_obj.get_notAfter().decode()
    anio = int(fecha[:4])
    mes = int(fecha[4:6])
    dia = int(fecha[6:8])

    if datetime(anio, mes, dia) >= datetime.now():
        print("Certificado vigente")
        return True
    else:
        print("Certificado no vigente, no puede firmar")
        return False


In [12]:
# Va a firmar el primer empleado

Certificado_1 = database.select(tabla = "users", 
                what = "Certificado", 
                where = "Nomina", 
                value = nomina_1)[0][0]

private_key_1 = "Ejemplo/privateA01750443.key"

VerificarVigencia(Certificado_1)

Match = check_associate_cert_with_private_key(Certificado_1, private_key_1)
print()

if Match:
    file_name = signVerify.gen_signature(private_key_1, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), file_1, nomina_1)
    print(f"Archivo firmado en {file_name}")

    print("\n Cargando la firma a la base de datos")
    database.cargar_firma(Doc_signed = file_name,
                     Hash = Hash_document(file_1).hexdigest(),
                     Nomina = nomina_1)
else:
    print("La llave privada no coincide con el certificado.\nNo puede firmar este documento.")

print("\n-------------------------------\n")

Certificado_2 = database.select(tabla = "users", 
                what = "Certificado", 
                where = "Nomina", 
                value = nomina_2)[0][0]

private_key_2 = "Ejemplo/privateA01752068.key"

VerificarVigencia(Certificado_2)

Match = check_associate_cert_with_private_key(Certificado_2, private_key_2)

if Match:

    file_name = signVerify.gen_signature(private_key_2, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), file_1, nomina_2)
    print(f"Archivo firmado en {file_name}")

    print("\n Cargando la firma a la base de datos")
    database.cargar_firma(Doc_signed = file_name,
                     Hash = Hash_document(file_1).hexdigest(),
                     Nomina = nomina_2)
else:
    print("La llave privada no coincide con el certificado.\nNo puede firmar este documento.")

Certificado vigente

Archivo firmado en Ejemplo/Doc_1.pdf_A01750443_20220421_082202.sign

 Cargando la firma a la base de datos

-------------------------------

Certificado vigente
Archivo firmado en Ejemplo/Doc_1.pdf_A01752068_20220421_082202.sign

 Cargando la firma a la base de datos


## Verificar firma

In [31]:
file_1 = "Ejemplo/Doc_1.pdf"
file_2 = "Ejemplo/Doc_2.pdf"

#Se emula que se suben los certificados

Certificado_1 = "Ejemplo/CertificadoA01750443.crt"
nomina_1 = "A01750443"
Certificado_2 = "Ejemplo/CertificadoA01752068.crt"

#Validando
f = database.select(tabla = "firmas", 
                what = "Doc_signed", 
                where = "Hash", 
                value = Hash_document(file_1).hexdigest(),
                where_2 = "Nomina",
                value_2 = nomina_1)[0][0]

if f != None:
    open("Ejemplo/temp.sign", "wb").write(f)

    result = signVerify.verify(Certificado_1, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), "Ejemplo/temp.sign", load = True)
    if result:
        print(f"Verificación exitosa. \nEl archivó fue firmado correctamente por {nomina_1}")
    else:
        print(f"Verificación fallida. \nEl archivó no firmado por {nomina_1}")

else:
    print(f"El archivo aún no cuenta con la firma de {nomina_1}")


Verification successful
Verificación fallida. 
El archivó no firmado por A01750443
